<a href="https://colab.research.google.com/github/fadel11-hub/Tim-Coba-Coba/blob/main/Final_Model_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# --- 1. Load dataset ---
df = pd.read_csv("/content/drive/MyDrive/Datathon/Tim/Dokumen/Dataset/darurat_aman_dataset.csv")  # ganti dengan file datasetmu
texts = df["text"].astype(str).values
labels = df["label"].values

# --- 2. Encode label ke angka ---
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# --- 3. Tokenisasi dan padding ---
tokenizer = Tokenizer(oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_length = 20  # panjang maksimum kalimat
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")

vocab_size = len(tokenizer.word_index) + 1  # +1 untuk padding
embedding_dim = 50

# --- 4. Split data ---
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, encoded_labels, test_size=0.2, random_state=42
)

# --- 5. Bangun model LSTM ---
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # binary classification
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# --- 6. Latih model ---
history = model.fit(
    X_train, y_train, epochs=10, batch_size=16,
    validation_data=(X_test, y_test)
)

# --- 7. Simpan model dan tokenizer ---
model.save("lstm_model.h5")
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.4571 - loss: 0.6958 - val_accuracy: 0.5366 - val_loss: 0.6929
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.4608 - loss: 0.6940 - val_accuracy: 0.4634 - val_loss: 0.6944
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.4765 - loss: 0.6931 - val_accuracy: 0.4634 - val_loss: 0.6941
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4391 - loss: 0.6962 - val_accuracy: 0.4634 - val_loss: 0.6945
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5120 - loss: 0.6929 - val_accuracy: 0.4634 - val_loss: 0.6949
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5027 - loss: 0.6934 - val_accuracy: 0.4634 - val_loss: 0.6934
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.4990 - loss: 0.6920 - val_accuracy: 0.5366 - val_loss: 0.6896
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.5902 - loss: 0.6845 - val_accuracy: 0.7561 - v

In [4]:
from tensorflow.keras.models import load_model
import pickle

# Load model & tokenizer
model = load_model("lstm_model.h5")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Kalimat baru
input_text = "saya menonton video"
seq = tokenizer.texts_to_sequences([input_text])
pad = pad_sequences(seq, maxlen=20, padding="post")

# Prediksi
pred_prob = model.predict(pad)[0][0]
pred_label = 1 if pred_prob >= 0.5 else 0
final_label = label_encoder.inverse_transform([pred_label])[0]

print("Kalimat:", input_text)
print("Probabilitas darurat:", pred_prob)
print("Prediksi Label:", final_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
Kalimat: saya menonton video
Probabilitas darurat: 0.008869636
Prediksi Label: aman
